#Content
1.   Prepreocess Images 
2.   Install StyleGAN2-ADA on your Google Drive
3.   Train a custom model
4.   Generate images from pre-trained model
5.   Latent space exploration


## Preprocess Images

In [ ]:
import PIL
import os
import os.path
from PIL import Image
from PIL import Image

f = r'/Users/huyihuan/Downloads/images'
for file in os.listdir(f):
    f_img = f+"/"+file
    img = Image.open(f_img).convert('RGB')
    img = img.resize((256, 256))
    img.save(f_img)

## Install StyleGAN2-ADA on your Google Drive

StyleGAN2-ADA only works with Tensorflow 1. Run the next cell before anything else to make sure we’re using TF1 and not TF2.


In [ ]:
%tensorflow_version 1.x
!nvidia-smi

TensorFlow 1.x selected.
Sat Jun 18 19:38:45 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                               

Then, mount Drive to the Colab notebook: 

In [ ]:
from google.colab import drive
from pathlib import Path

content_path = Path('/').absolute() / 'content'
drive_path = content_path / 'drive'
drive.mount(str(drive_path))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Finally, run this cell to install StyleGAN2-ADA on the Drive. If it is already installed the repository, it will skip the installation process and only check for updates. If it haven’t installed it, it will install all the necessary files. Beside, **in**, **out**, **datasets** and **training** folders are generated for data storage. 

In [ ]:
stylegan2_repo_url  = 'https://github.com/dvschultz/stylegan2-ada' # or https://github.com/NVlabs/stylegan2-ada
project_path        = drive_path / 'MyDrive' / 'StyleGAN2-ADA'
stylegan2_repo_path = project_path / 'stylegan2-ada'

# Create project folder if inexistant
if not project_path.is_dir():
    %mkdir "{project_path}"
%cd "{project_path}"

for dir in ['in', 'out', 'datasets', 'training']:
    if not (project_path / dir).is_dir():
        %mkdir {dir}
if not (project_path / 'datasets' / 'source').is_dir():
    %mkdir "{project_path / 'datasets' / 'source'}"

# Download StyleGAN2-ada
!git config --global user.name "GANightSky"
!git config --global user.email "NightSky@gan.com"
if stylegan2_repo_path.is_dir():
    !git -C "{stylegan2_repo_path}" fetch origin
    !git -C "{stylegan2_repo_path}" checkout origin/main -- *.py
else:
    print("Install StyleGAN2-ADA")
    !git clone {stylegan2_repo_url}

/content/drive/MyDrive/StyleGAN2-ADA


## Train a custom model

In [ ]:
dataset_name = 'pokemon_500'
datasets_source_path = project_path / 'datasets' / 'source' / (dataset_name + '.zip')
if datasets_source_path.is_file():
    print("Dataset ready for import.")
else:
    print('Upload your images dataset as {}'.format(datasets_source_path))

Dataset ready for import.


Run this cell to download the zipped dataset from Drive and unzip it in the local session.

In [ ]:
local_dataset_path =project_path / 'datasets' / 'source' #content_path / 'dataset' 
if not local_dataset_path.is_dir():
    print("Importing dataset...")
    %mkdir "{local_dataset_path}"
    %cp -a "{project_path / 'datasets' / 'source' / (dataset_name + '.zip')}" "{local_dataset_path}"
    print("Zip file succesfuly imported")
else:
    print('Zip file allready imported')

import zipfile
with zipfile.ZipFile(str(local_dataset_path / (dataset_name + '.zip')), 'r') as zip_ref:
    zip_ref.extractall(str(local_dataset_path))
print('Extraction completed')

Zip file allready imported
Extraction completed


### Convert dataset to .tfrecords

Next, we need to convert our image dataset to a format that StyleGAN2-ADA can read:`.tfrecords`.

This can take a while.

In [ ]:
local_images_path = project_path / 'datasets' / 'source' / 'images'
local_dataset_path /= 'tfr'

if (local_dataset_path).is_dir():
    print('\N{Heavy Exclamation Mark Symbol} Dataset already created \N{Heavy Exclamation Mark Symbol}')
    print('Delete current dataset folder ({}) to regenerate tfrecords.'.format(local_dataset_path))
else:
    %mkdir "{local_dataset_path}"
    !python "{stylegan2_repo_path / 'dataset_tool.py'}" create_from_images \
        "{local_dataset_path}" "{local_images_path}"

Loading images from "/content/drive/MyDrive/StyleGAN2-ADA/datasets/source/images"
Creating dataset "/content/drive/MyDrive/StyleGAN2-ADA/datasets/source/tfr"
Added 809 images.


There are numerous arguments to tune the training of the model. Here are some parameters I played with:


*   *mirror:* Should the images be mirrored vertically?
*   *mirrory:* Should the images be mirrored horizontally?
*   *snap:* How often should the model generate image samples and a network pickle (.pkl file)?
*   *resume:* Network pickle to resume training from?


In [ ]:
#!python "{stylegan2_repo_path / 'train.py'}" --help

In [ ]:
#!pip install -U numpy==1.18.5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
training_path = project_path / 'training' / dataset_name
if not training_path.is_dir():
    %mkdir "{training_path}"

#how often should the model generate samples and a .pkl file
snapshot_count = 1000
#should the images be mirrored left to right?
mirrored = True
#should the images be mirrored top to bottom?
mirroredY = False
#metrics? 
metric_list = None
#augments
augs = 'bgc'
#train time
total_kimgs =  10
#train transfer 
resume_from = 'ffhq256'

!python "{stylegan2_repo_path / 'train.py'}" --outdir="{training_path}" \
    --data="{local_dataset_path}" --resume="{resume_from}" \
    --snap={snapshot_count} --augpipe={augs} \
    --mirror={mirrored} --mirrory={mirroredY} \
    --metrics={metric_list} #\
    #--total_kimg = {total_kimgs}
    #--dry-run

tcmalloc: large alloc 4294967296 bytes == 0x7e7a000 @  0x7fc9b18f6001 0x7fc9aeae07b5 0x7fc9aeb44c00 0x7fc9aeb46a9f 0x7fc9aebdd078 0x593835 0x548c51 0x5127f1 0x549e0e 0x4bca8a 0x532b86 0x594a96 0x548cc1 0x5127f1 0x549576 0x4bca8a 0x5134a6 0x549576 0x4bca8a 0x5134a6 0x549e0e 0x4bca8a 0x5134a6 0x593dd7 0x5118f8 0x549576 0x604173 0x5f5506 0x5f8c6c 0x5f9206 0x64faf2
tcmalloc: large alloc 4294967296 bytes == 0x7fc7dedc4000 @  0x7fc9b18f41e7 0x7fc9aeae0631 0x7fc9aeb44cc8 0x7fc9aeb44f87 0x7fc9aebdcf58 0x593835 0x548c51 0x5127f1 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x548ae9 0x5127f1 0x593dd7 0x5118f8 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x549e0e
tcmalloc: large alloc 4294967296 bytes == 0x7fc6dddc2000 @  0x7fc9b18f41e7 0x7fc9aeae0631 0x7fc9aeb44cc8 0x7fc9aeb44f87 0x7fc971e25235 0x7fc9717a8792 0x7fc9717a8d42 0x7fc971761aee 0x59371f 0x548c51 0x51566f 0x593dd7 0x511e2c 0x549e0e 0x4bcb19 0x5134a6 0x549576 0x593f

## Generate images from pre-trained model


In [ ]:
%pip install opensimplex
!python "{stylegan2_repo_path / 'generate.py'}" generate-images --help 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)3694592/45929032 bytes (8.0%)7667712/45929032 bytes (16.7%)12009472/45929032 bytes (26.1%)16187392/45929032 bytes (35.2%)20365312/45929032 bytes (44.3%)24576000/45929032 bytes (53.5%)28868608/45929032 bytes (62.9%)33062912/45929032 bytes (72.0%)37158912/45929032 bytes (80.9%)41508864/45929032 bytes (90.4%)45703168/45929032 byte

In [ ]:
from numpy import random
seed_init = random.randint(10000)
nbr_images = 6

generation_from = '/content/drive/MyDrive/StyleGAN2-ADA/training/pokemon_500/network-snapshot-000330.pkl'

!python "{stylegan2_repo_path / 'generate.py'}" generate-images \
    --outdir="{project_path / 'out'}" --trunc=0.7 \
    --seeds={seed_init}-{seed_init+nbr_images-1} --create-grid \
    --network={generation_from}

Loading networks from "/content/drive/MyDrive/StyleGAN2-ADA/training/pokemon_500/network-snapshot-000330.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Compiling... Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Compiling... Loading... Done.
Generating image for seed 5790 (0/6) ...
Generating image for seed 5791 (1/6) ...
Generating image for seed 5792 (2/6) ...
Generating image for seed 5793 (3/6) ...
Generating image for seed 5794 (4/6) ...
Generating image for seed 5795 (5/6) ...
Generating image grid...


## Latent space exploration

In [ ]:
%pip install opensimplex
!python "{stylegan2_repo_path / 'generate.py'}" generate-latent-walk --help 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
usage: generate.py generate-latent-walk [-h] --network NETWORK_PKL
                                        [--trunc TRUNCATION_PSI]
                                        [--walk-type WALK_TYPE]
                                        [--frames FRAMES] [--fps FRAMERATE]
                                        [--seeds SEEDS] [--npys NPYS]
                                        [--save_vector] [--diameter DIAMETER]
                                        [--start_seed START_SEED]
                                        [--outdir DIR]

optional arguments:
  -h, --help            show this help message and exit
  --network NETWORK_PKL
                        Network pickle filename
  --trunc TRUNCATION_PSI
                        Truncation psi (default: 0.5)
  --walk-type WALK_TYPE
                        Type of walk (default: line)
  --frames FRAMES       Frame count (default: 240
  --f

### Explore through multiple seeds

In [ ]:
from numpy import random
walk_types = ['line', 'sphere']
latent_walk_path = project_path / 'out' / 'latent_walk'
if not latent_walk_path.is_dir():
    %mkdir "{latent_walk_path}"

explored_network = '/content/drive/MyDrive/StyleGAN2-ADA/training/pokemon_500/network-snapshot-000330.pkl'#'ffhq1024'

seeds = [random.randint(10000) for i in range(3)]
print("Base seeds:", seeds)
!python "{stylegan2_repo_path / 'generate.py'}" generate-latent-walk --network="{explored_network}" \
    --outdir="{latent_walk_path}" --trunc=0.5 --walk-type="{walk_types[0]}" \
    --seeds={','.join(map(str, seeds))} --frames {len(seeds)*30}

Base seeds: [8474, 6171, 3046]
Loading networks from "/content/drive/MyDrive/StyleGAN2-ADA/training/pokemon_500/network-snapshot-000330.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Loading... Done.
Generating image for step 0/92 ...
Generating image for step 1/92 ...
Generating image for step 2/92 ...
Generating image for step 3/92 ...
Generating image for step 4/92 ...
Generating image for step 5/92 ...
Generating image for step 6/92 ...
Generating image for step 7/92 ...
Generating image for step 8/92 ...
Generating image for step 9/92 ...
Generating image for step 10/92 ...
Generating image for step 11/92 ...
Generating image for step 12/92 ...
Generating image for step 13/92 ...
Generating image for step 14/92 ...
Generating image for step 15/92 ...
Generating image for step 16/92 ...
Generating image for step 17/92 ...
Generating image for step 18/92 ...
Generating image for step 19/92 ...
Generating image

### Explore around a single seed

In [ ]:
from numpy import random
walk_types = ['noiseloop', 'circularloop']

latent_walk_path = project_path / 'out' / 'latent_walk'
if not latent_walk_path.is_dir():
    %mkdir "{latent_walk_path}"

#explored_network = 'f256'

start_seed = random.randint(10000)

!python "{stylegan2_repo_path / 'generate.py'}" generate-latent-walk --network="{explored_network}" \
    --outdir="{latent_walk_path}" --trunc=0.4 --walk-type=="{walk_types[1]}" \
    --start_seed="{start_seed}" --diameter=20 --frames {24*5}

Loading networks from "/content/drive/MyDrive/StyleGAN2-ADA/training/pokemon_500/network-snapshot-000330.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Loading... Done.
Traceback (most recent call last):
  File "/content/drive/MyDrive/StyleGAN2-ADA/stylegan2-ada/generate.py", line 814, in <module>
    main()
  File "/content/drive/MyDrive/StyleGAN2-ADA/stylegan2-ada/generate.py", line 809, in main
    func(**kwargs)
  File "/content/drive/MyDrive/StyleGAN2-ADA/stylegan2-ada/generate.py", line 470, in generate_latent_walk
    generate_latent_images(points, truncation_psi, outdir, save_vector,'frame', seed_out, framerate)
UnboundLocalError: local variable 'points' referenced before assignment
